In [ ]:
import os
import sys
import h5py
import argparse
import numpy as np
from collections import Counter

# 3rd party imports
from sklearn import manifold
from keras.models import Model
from keras import backend as K
from IPython.display import Image

# ML4CVD Imports
sys.path.append("../ml4cvd")
from arguments import parse_args
from tensor_generators import test_train_valid_tensor_generators
from recipes import train_shallow_model, train_multimodal_multitask, test_multimodal_multitask
from models import make_multimodal_to_multilabel_model, train_model_from_generators, make_hidden_layer_model

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

In [ ]:
sys.argv = ['train', 
            '--tensors', '/mnt/disks/survey-tensors2/2019-03-25/', 
            '--input_tensors', 'categorical-phenotypes-32', 
            '--output_tensors', 'allergic_rhinitis', 'anxiety', 'asthma', 'atrial_fibrillation_or_flutter', 'back_pain', 
            'breast_cancer', 'cardiac_surgery', 'cervical_cancer', 'colorectal_cancer', 'coronary_artery_disease_hard', 
            'coronary_artery_disease_intermediate', 'coronary_artery_disease_soft', 'death', 'diabetes_all', 
            'diabetes_type_1', 'diabetes_type_2', 'enlarged_prostate', 'heart_failure', 'hypertension', 'lung_cancer', 
            'migraine', 'myocardial_infarction', 'osteoporosis', 'skin_cancer', 'stroke',
            '--id', 'cat32_mlp',
            '--epochs', '1',
            '--training_steps', '100', 
            '--validation_steps', '2', 
            '--batch_size', '512',
            '--model_file', '/mnt/ml4cvd/projects/jamesp/data/models/cat32_mlp.hd5'
           ]
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,
                                                                                       args.tensor_maps_out,
                                                                                       args.tensors, args.batch_size,
                                                                                       args.valid_ratio,
                                                                                       args.test_ratio,
                                                                                       args.icd_csv,
                                                                                       args.balance_by_icds)
model = make_multimodal_to_multilabel_model(args.model_file, args.model_layers, args.model_freeze,
                                                args.tensor_maps_in, args.tensor_maps_out, args.activation,
                                                args.dense_layers, args.dropout, args.mlp_concat, args.conv_layers,
                                                args.max_pools, args.res_layers, args.dense_blocks, args.block_size,
                                                args.conv_bn, args.conv_x, args.conv_y, args.conv_z, args.conv_dropout,
                                                args.conv_width, args.u_connect, args.pool_x, args.pool_y, args.pool_z, args.padding,
                                                args.learning_rate)

In [ ]:
layer_name = 'dense_1'
d1 = model.get_layer(layer_name)
w1 = d1.get_weights()
rev_cm =  dict((v,k) for k,v in args.tensor_maps_in[0].channel_map.items())
for w in w1:
    print(w.shape)
embed_model = make_hidden_layer_model(model, layer_name)
embed_model.summary()
test_data, test_labels, test_paths = next(generate_test)
print(list(test_data.keys()))
x_embed = embed_model.predict(test_data)
y_pred = model.predict(test_data)

In [ ]:
categorical_labels = ['Genetic-sex_Female_0_0', 'hypertension', 'coronary_artery_disease', 'Handedness-chiralitylaterality_Righthanded_0_0']
continuous_labels = ['22200_Year-of-birth_0_0|34_Year-of-birth_0_0', '21001_Body-mass-index-BMI_0_0', '1070_Time-spent-watching-television-TV_0_0', '102_Pulse-rate-automated-reading_0_0', '1488_Tea-intake_0_0', '21002_Weight_0_0']
label_dict = {k: np.zeros((len(test_paths))) for k in categorical_labels + continuous_labels}
for i, tp in enumerate(test_paths):
    hd5 = h5py.File(tp, 'r')
    #print(list(hd5['continuous'].keys()))
    for k in categorical_labels:
        if k in hd5['categorical']:
            label_dict[k][i] = 1
        elif k in hd5 and hd5[k][0] == 1:
            label_dict[k][i] = 1
    for mk in continuous_labels:
        for k in mk.split('|'):
            if k in hd5['continuous']:
                label_dict[mk][i] = hd5['continuous'][k][0]

print(list(label_dict.keys()))
print(label_dict['22200_Year-of-birth_0_0|34_Year-of-birth_0_0'])

In [ ]:
n_components = 2
max_rows = 10
(fig, subplots) = plt.subplots(min(max_rows, len(label_dict)), 3, figsize=(16, max_rows*4))
perplexities = [8, 18, 35]

p2y = {}
for i, perplexity in enumerate(perplexities):
    tsne = manifold.TSNE(n_components=n_components, init='random', random_state=0, perplexity=perplexity)
    p2y[perplexity] = tsne.fit_transform(x_embed)

j = -1
for k in label_dict:
    j += 1
    if j == max_rows:
        break
    if k in categorical_labels:
        red = label_dict[k] == 1.0
        green = label_dict[k] != 1.0
    elif k in continuous_labels:
        colors = label_dict[k]       
    print('process key:', k)
    for i, perplexity in enumerate(perplexities):
        ax = subplots[j, i]
        ax.set_title(k+", Perplexity=%d" % perplexity)
        if k in categorical_labels:
            ax.scatter(p2y[perplexity][green, 0], p2y[perplexity][green, 1], c="g")
            ax.scatter(p2y[perplexity][red, 0], p2y[perplexity][red, 1], c="r")
            ax.legend(['no_'+k, k], loc='lower left')
        elif k in continuous_labels:
            points = ax.scatter(p2y[perplexity][:, 0], p2y[perplexity][:, 1], c=colors, cmap='jet') 
            if i == len(perplexities)-1:
                fig.colorbar(points, ax=ax)
                
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.axis('tight')
plt.show()